In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 8.8 MB/s 
     |████████████████████████████████| 140 kB 65.6 MB/s 
     |████████████████████████████████| 358 kB 68.9 MB/s 
     |████████████████████████████████| 58 kB 7.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


# `실종동물 스크래핑`

### [긴급] 실종 동물 스크래핑

In [ ]:
def emergency_scraping(animal_url, wd):
  try:
    emergency_animals = []

    for t in range(1, 100):
      for i in range(1, 4):
        emergency_url = wd.find_element(By.XPATH, '/html/body/div[4]/div/div/div/div[%d]/div[1]/div[%d]/div/a' %(t,i) ).get_attribute('href')
        emergency_animals.append(emergency_url)

        
  except:
    pass
  return emergency_animals


## [실종] 실종 동물 스크래핑

In [ ]:
def missing_animals_url(animal, wd):
  try: 
    missing_animals= []
    totalpages = wd.find_element(By.XPATH, '/html/body/div[6]/div/div/div/p').text
    page = totalpages.split(" ")
    last = int(page[6]) + 1
    
    for p in range(1, last):
      url = 'http://www.angel.or.kr/index.php?code=%s&page=%d' %(animal, p)
      wd.get(url)
      for t in range(1, 6):
        for i in range(1, 5):
          if(p==1):
            missing = wd.find_element(By.XPATH, '/html/body/div[5]/div/div/div/div[%d]/div[1]/div[%d]/h3' %(t,i) ).text
          else:
            missing = wd.find_element(By.XPATH, '/html/body/div[4]/div/div/div/div[%d]/div[1]/div[%d]/h3' %(t,i) ).text

          if("실종" in missing):  
            if(p==1):
              missing_url = wd.find_element(By.XPATH, '//html/body/div[5]/div/div/div/div[%d]/div[1]/div[%d]/div/a' %(t,i) ).get_attribute('href')
            else:
              missing_url = wd.find_element(By.XPATH, '//html/body/div[4]/div/div/div/div[%d]/div[1]/div[%d]/div/a' %(t,i) ).get_attribute('href')
            missing_animals.append(missing_url)
  except: 
    pass


  return missing_animals

## 실종 동물 상세 정보

In [ ]:
def animals_info_scraping(animal_info_url, wd):

  animal = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/table/tbody/tr[1]/td[2]').text
  info = animal.split('/')
  find = info[len(info)-4]
  gender = info[len(info)-3]
  age = info[len(info)-2]
  name = info[len(info)-1]
  missingDay = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/table/tbody/tr[2]/td[2]').text
  location = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/table/tbody/tr[3]/td[2]').text
  description = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/table/tbody/tr[5]/td[2]').text
  description = description.replace("\n", " ")
  description = description.replace("\t", " ")

  # 이미지 클릭해 이미지로 이동
  try:
    img_url = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/div[3]/div[1]/div[1]/div/a').get_attribute('href')
    wd.get(img_url)
    img = wd.find_element(By.XPATH, '/html/body/img').get_attribute('src')
  except:
    img = ''
    pass


  return [find, gender, age, name, missingDay, location ,description, img]

## `URL_스크래핑` : 강아지

In [ ]:
def dog_url_scraping():

  wd = webdriver.Chrome('chromedriver', options= chrome_options)
  wd.implicitly_wait(3)
  
  dogs_url = 'http://www.angel.or.kr/index.php?code=dog'
  wd.get(dogs_url)
  
  dogs_df = pd.DataFrame(columns=("Find", "Gender", "Age", "Name", "MissingDay", "Location", "Description", "IMG"))

  emergency_dogs = emergency_scraping(dogs_url, wd)
  
  dogs_idx = 0
  for dog_url in emergency_dogs:
    wd.get(dog_url)
    dogs_df.loc[dogs_idx] = animals_info_scraping(dog_url, wd)
    dogs_idx += 1

  wd.get(dogs_url)
  missing_dogs_url = missing_animals_url('dog', wd)
  
  for dog_url in missing_dogs_url:
    wd.get(dog_url)
    dogs_df.loc[dogs_idx] = animals_info_scraping(dog_url, wd)
    dogs_idx += 1

  return dogs_df
  


In [ ]:
dog_df = dog_url_scraping()

In [ ]:
dog_df

## `URL_스크래핑` : 고양이

In [ ]:
def cat_url_scraping():
  wd = webdriver.Chrome('chromedriver', options= chrome_options)
  wd.implicitly_wait(3)
  cats_url = 'http://www.angel.or.kr/index.php?code=cat'
  
  wd.get(cats_url)

  cats_df = pd.DataFrame(columns=("Find", "Gender", "Age", "Name", "MissingDay", "Location", "Description", "IMG"))

  emergency_cats = emergency_scraping(cats_url, wd)

  cats_idx = 0
  for cat_url in emergency_cats:
    wd.get(cat_url)
    cats_df.loc[cats_idx] = animals_info_scraping(cat_url, wd)
    cats_idx += 1

  wd.get(cats_url)
  missing_cats_url = missing_animals_url('cat', wd)

  for cat_url in missing_cats_url:
    wd.get(cat_url)
    cats_df.loc[cats_idx] = animals_info_scraping(cat_url, wd)
    cats_idx += 1

  return cats_df

In [ ]:
cats_df = cat_url_scraping()

In [ ]:
cats_df

## `URL_스크래핑` : 그외 동물

In [ ]:
def other_url_scraping():
  wd = webdriver.Chrome('chromedriver', options= chrome_options)
  wd.implicitly_wait(3)
  others_url = 'http://www.angel.or.kr/index.php?code=other'

  wd.get(others_url)

  emergency_others = emergency_scraping(others_url, wd)

  others_df = pd.DataFrame(columns=("Find", "Gender", "Age", "Name", "MissingDay", "Location", "Description", "IMG"))

  others_idx = 0
  for other_url in emergency_others:
    wd.get(other_url)
    others_df.loc[others_idx] = animals_info_scraping(other_url, wd)
    others_idx += 1

  wd.get(others_url)
  missing_others_url = missing_animals_url('other', wd)

  for other_url in missing_others_url:
    wd.get(other_url)
    others_df.loc[others_idx] = animals_info_scraping(other_url, wd)
    others_idx += 1
  return others_df


In [ ]:
others_df = other_url_scraping()

In [ ]:
others_df

In [ ]:
dogs_df = dog_url_scraping()
cats_df = cat_url_scraping()
others_df = other_url_scraping()

In [ ]:
dogs_df

,Find,Gender,Age,Name,MissingDay,Location,Description,IMG
0,포메라니언,암컷,6살,이름(이브),2022-09-13,전라북도 김제시 용지면 용지면사무소 근처,"전체적으로 황색인데 가슴,배쪽은 흰털 사람 엄청 좋아함 눈 주위가 하얗고 아이라인이...",http://www.angel.or.kr/report/dog/upImg/166313...
1,이탈리언 그레이하운드,암컷,6살,이름(세모),2022-09-01,충청남도 아산시 아산 은행나무길 / 삽교회전교차로,얼룩무늬 그레이하운드과 휘펫 노락색옷입고있습니다 사슴같이 말랐습니다,http://www.angel.or.kr/report/dog/upImg/166313...
2,진돗개,암컷,1살미만,이름(알콩이),2022-09-13,경기도 남양주시 중흥s클래스 2206동 주차장,"겁많음, 성인남자트라우마, 소리예민, 큰움직임 예민",http://www.angel.or.kr/report/dog/upImg/166305...
3,믹스견(진도 믹스 아이보리색상),수컷,5살,이름(까불이),2022-9-10,경기도 고양시 일산동구 성석동 빅토리아 골프연습장 인근,몸 길이 80cm 중성화 안됨 심장사상충 약 복용으로 입과 코 주변이 하얀색임 목줄...,http://www.angel.or.kr/report/dog/upImg/166291...
4,믹스견(말티즈+시츄),암컷,9살,이름(코코),2022-9-2,경기도 수원시 팔달구 수원월드메르디앙- 수원월드컵 경기장,속눈썹이 엄청 길어요,http://www.angel.or.kr/report/dog/upImg/166226...
...,...,...,...,...,...,...,...,...
5336,"믹스견(시츄 비숑, 짙은회색단발귀)",수컷,10살,이름(둥삼),2016-7-30,경기도 파주시 운정 해솔마을 6단지 팻샵,"마른체형,다리가 김 , 5kg이하 짙은회색 단발귀",
5337,믹스견(발바리),암컷,3살,이름(영인이),2016-8-02,충청남도 아산시 권곡동 화성빌라 다동 근처,잃어버리기 전에 털을 깍았습니다...,
5338,미니어처 푸들,암컷,5살,이름(뿡이),2016-7-28,경기도 이천시 백사면 조읍리 한솔아파트단지내,연한갈색푸들이고 털이 많이자라있음 목에 흰털이있고 귀와 꼬리가 짧음,
5339,진돗개,수컷,1살미만,이름(똘똘이),2016-8-8,경상북도 포항시 남구 포항시 남구 오천읍 원리1343번지 부근,나이는 4개월정도 되었습니다.수컷 입니다.,


In [ ]:
cats_df

,Find,Gender,Age,Name,MissingDay,Location,Description,IMG
0,믹스묘,수컷,1살,이름(나비),2022-09-10,경기도 평택시 평택헌잭방 부근,목과 뒷다리사이에 동그랗게 흰털이 있음 애교많고 겁이 많음 목걸이 앞면에 까미 나비...,http://www.angel.or.kr/report/cat/upImg/166323...
1,벵갈 캣,암컷,6살,이름(버디),2022-8-26,서울특별시 용산구 유엔빌리지 힐미드빌라,4키로 정도 나가고 소심하고 겁이 많습니다 호피무늬,http://www.angel.or.kr/report/cat/upImg/166182...
2,코리아 쇼트 헤어,암컷,2살,이름(김만두),2022-8-27,인천광역시 계양구 귤현동 굴재길 3-3 현대렉스힐 부근,"삼색 고양이 보는 사람 기준 코 왼쪽에 검정 무늬, 왼쪽 다리 옆에 갈색 무늬, 오...",http://www.angel.or.kr/report/cat/upImg/166159...
3,코리아 쇼트 헤어,암컷,4살,이름(김애용),2022-08-15,경상북도 구미시 지산동 금오여자고등학교 맞은 편 논 부근 에서 실종,코리안 숏헤어 치즈냥 오른쪽 눈이 없고 얼굴에 아아라인처럼 눈선을 따라 진한 갈색 ...,http://www.angel.or.kr/report/cat/upImg/166114...
4,코리아 쇼트 헤어,수컷,5살,이름(수염이),2022-07-05,서울특별시 강서구 푸주옥 설렁탕 부근,흰+검 젖소냥이 코 밑에 검정콧물무늬 중성화된 수컷냥이 8키로로 덩치가 있는 편이었습니다,http://www.angel.or.kr/report/cat/upImg/165919...
...,...,...,...,...,...,...,...,...
2379,샴,암컷,3살,이름(땅콩),2016-08-13,서울특별시 노원구 상계2동 대게나라 부근,샴 고양이 입니다. 아직 중성화 수술은 안했습니다. 사진을 참조하여주시길 바랍니다.,http://www.angel.or.kr/report/cat/upImg/147126...
2380,아메리칸 쇼트헤어,수컷,2살,이름(겁식이),2016-8-11,경상남도 창원시 마산회원구 마산 2부두 (어시장 아이파크 부근),"노란털 +갈색줄, 가슴 발 하얀색, 눈 노란색, 은목걸이 착용( 전화번호 있어요)",
2381,러시안 블루,수컷,2살,이름(타비),2016-8-14,인천광역시 남동구 신일해피트리 부근,뚱뚱하며 몸통만 털이짧음 겁이 매우많으며 애교가 엄청많음 2살 중성화x,
2382,코리아 쇼트 헤어,암컷,1살미만,이름(초코),2016-5-12,부산광역시 동래구 안락동 석천아파트 낙민동 벽산아파트 부근,자주색 방울목걸이 착용 꼬리가 꺾여있음 15년11월출생,http://www.angel.or.kr/report/cat/upImg/147109...


In [ ]:
others_df

,Find,Gender,Age,Name,MissingDay,Location,Description,IMG
0,앵무새,암컷,1살,이름(일랑이),2022-8-22,대구광역시 수성구 대구 범어동 수성구청역점 서브웨이 뒷쪽 주택단지,블루 퀘이커 앵무새 비둘기보다 많이 작고 참새보다 커요 윙컷을 안한 상태이고 부리쪽...,http://www.angel.or.kr/report/other/upImg/1661...
1,앵무새,암컷,1살,이름(찬송이),2022-8-12,제주특별자치도 제주시 연동 한라유치원부근 산책로,초록색퀘이커예요 빵.국수좋아해요 겁이많아서 물수도있어요,http://www.angel.or.kr/report/other/upImg/1660...
2,앵무새,수컷,1살미만,이름(산초),2022-08-04,"전라북도 군산시 장자도에서 실종. 선유도,무녀도,신시도 발견가능성 높음","회색앵무, 발목링을 하고있습니다. 큰소릴 내거나 섣불리 잡으려 마시고 빠른 연락부탁...",http://www.angel.or.kr/report/other/upImg/1659...
3,조류(모란앵무),성별모름,3살,이름(수박나나),2022-9-14,울산광역시 북구 북구 명촌성당 부근,"한 마리는 얼굴이 빨갛고 몸은 녹색, 한 마리는 배쪽은 하얗고 등쪽은 연노랑~~두 ...",http://www.angel.or.kr/report/other/upImg/1663...
4,조류(쟈딘앵무새),암컷,17살,이름(코난),2022-9-12,경상남도 김해시 건영아파트,초록색과 검정색이 섞여있구요. 머리에는 살짝 빨간색이 있습니다. 성인남자 손보다 크...,http://www.angel.or.kr/report/other/upImg/1662...
...,...,...,...,...,...,...,...,...
335,앵무새,수컷,1살,이름(시월),2016-08-31,대전광역시 동구 자양동 196-11,"왕관앵무 루티노 이며, 윙트리밍(윙컷)이 전혀 되지 않은 아이입니다. 발견시 잡으려...",http://www.angel.or.kr/report/other/upImg/1472...
336,앵무새,성별모름,2살,이름(바로),2016-8-28,경기도 안양시 동안구 대륭테크노 근처,초록색 앵무새입니다,http://www.angel.or.kr/report/other/upImg/1472...
337,앵무새,암컷,4살,이름(왕관앵무새),2016-8-24,경기도 고양시 덕양구 행신동 서정마을 1단지,왕관앵무 회색새입니다 사람 잘따르고 다리가 힘이조금 약해요,http://www.angel.or.kr/report/other/upImg/1472...
338,앵무새,암컷,4살,이름(루비),2016-8-16,경기도 수원시 장안구 정자1동 동신아파트 104동1단지앞,왕관앵무새이며노란색이고볼에는연지곤지처럼생겼습니다.낯을많이가려서잡으려하면날라갈수도있습니다,http://www.angel.or.kr/report/other/upImg/1471...


In [ ]:
dogs_df_clone = dogs_df
cats_df_clone = cats_df
others_df_clone = others_df

merge_df = pd.concat([dogs_df_clone, cats_df_clone])


In [ ]:
merge_df = pd.concat([merge_df, others_df_clone])
merge_df 

,Find,Gender,Age,Name,MissingDay,Location,Description,IMG
0,포메라니언,암컷,6살,이름(이브),2022-09-13,전라북도 김제시 용지면 용지면사무소 근처,"전체적으로 황색인데 가슴,배쪽은 흰털 사람 엄청 좋아함 눈 주위가 하얗고 아이라인이...",http://www.angel.or.kr/report/dog/upImg/166313...
1,이탈리언 그레이하운드,암컷,6살,이름(세모),2022-09-01,충청남도 아산시 아산 은행나무길 / 삽교회전교차로,얼룩무늬 그레이하운드과 휘펫 노락색옷입고있습니다 사슴같이 말랐습니다,http://www.angel.or.kr/report/dog/upImg/166313...
2,진돗개,암컷,1살미만,이름(알콩이),2022-09-13,경기도 남양주시 중흥s클래스 2206동 주차장,"겁많음, 성인남자트라우마, 소리예민, 큰움직임 예민",http://www.angel.or.kr/report/dog/upImg/166305...
3,믹스견(진도 믹스 아이보리색상),수컷,5살,이름(까불이),2022-9-10,경기도 고양시 일산동구 성석동 빅토리아 골프연습장 인근,몸 길이 80cm 중성화 안됨 심장사상충 약 복용으로 입과 코 주변이 하얀색임 목줄...,http://www.angel.or.kr/report/dog/upImg/166291...
4,믹스견(말티즈+시츄),암컷,9살,이름(코코),2022-9-2,경기도 수원시 팔달구 수원월드메르디앙- 수원월드컵 경기장,속눈썹이 엄청 길어요,http://www.angel.or.kr/report/dog/upImg/166226...
...,...,...,...,...,...,...,...,...
335,앵무새,수컷,1살,이름(시월),2016-08-31,대전광역시 동구 자양동 196-11,"왕관앵무 루티노 이며, 윙트리밍(윙컷)이 전혀 되지 않은 아이입니다. 발견시 잡으려...",http://www.angel.or.kr/report/other/upImg/1472...
336,앵무새,성별모름,2살,이름(바로),2016-8-28,경기도 안양시 동안구 대륭테크노 근처,초록색 앵무새입니다,http://www.angel.or.kr/report/other/upImg/1472...
337,앵무새,암컷,4살,이름(왕관앵무새),2016-8-24,경기도 고양시 덕양구 행신동 서정마을 1단지,왕관앵무 회색새입니다 사람 잘따르고 다리가 힘이조금 약해요,http://www.angel.or.kr/report/other/upImg/1472...
338,앵무새,암컷,4살,이름(루비),2016-8-16,경기도 수원시 장안구 정자1동 동신아파트 104동1단지앞,왕관앵무새이며노란색이고볼에는연지곤지처럼생겼습니다.낯을많이가려서잡으려하면날라갈수도있습니다,http://www.angel.or.kr/report/other/upImg/1471...


In [ ]:
merge_df.to_csv('missing_animal.csv')